In [ ]:
pip install pymc

In [1]:
import pandas as pd
import numpy as np
import pymc as pm
import matplotlib.pyplot as plt

In [2]:
# Data Preparation
df = pd.read_csv('Eartquakes-1990-2023.csv')


df['state'] = df['state'].astype('category')
state_idx = df['state'].cat.codes.values
states = df['state'].cat.categories
n_states = len(states)

In [4]:
with pm.Model() as model:
    # Priors for unknown model parameters
    alpha = pm.Normal('alpha', mu=0, sigma=10, shape=n_states)
    beta = pm.Normal('beta', mu=0, sigma=10)

    # Expected value of outcome
    mu = alpha[state_idx] + beta * df['magnitudo']

    # Likelihood (sampling distribution) of observationsw
    Y_obs = pm.Normal('Y_obs', mu=mu, sigma=1, observed=df['magnitudo'])

    # Inference
    trace = pm.sample(500, tune=500, chains=1, return_inferencedata=True)

# Plotting the results
pm.plot_trace(trace)
plt.show()

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Sequential sampling (1 chains in 1 job)
NUTS: [alpha, beta]


In [9]:
df.columns

Index(['time', 'place', 'status', 'tsunami', 'significance', 'data_type',
       'magnitudo', 'state', 'longitude', 'latitude', 'depth', 'date'],
      dtype='object')